In [8]:
import os 
import os.path as osp
import numpy as np 
import cv2
import matplotlib.pyplot as plt
import matplotlib.image as mpimg
import matplotlib.patches as patches
from mpl_toolkits.mplot3d import Axes3D
import random
import open3d as o3d
import open3d.core as o3c
import json
import subprocess
import pickle
from  plyfile import PlyData
import torch
from tqdm import tqdm
import torch.multiprocessing as mp
from concurrent.futures import ThreadPoolExecutor
#the other imports from the local stuff
import sys

ws_dir = '/local/home/ekoller/BT'
print(ws_dir)
sys.path.append(ws_dir)
from utils import scan3r,visualisation

/local/home/ekoller/BT


In [12]:
#reading in the necessary data
data_dir ='/local/home/ekoller/R3Scan'
scenes_dir = '/local/home/ekoller/R3Scan/scenes'
scan_id= "10b1795b-3938-2467-88fe-b10cad8913f8" #is reference scan  since it is a reference scan everything shouls be correctly hit
frame_number = "000086"
img_width = 960
img_height = 540

curr_scan_id = "10b1795b-3938-2467-88fe-b10cad8913f8" #is ref
new_scan_id =  "9c27de56-6184-2cda-8196-591957b6387d" #is rescan

#the original meshes are given in the file  'labels.instances.annotated.v2.ply'


device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
#print(torch.cuda.is_available() )



This code segment does the following: given a mesh of the scene, create the camera rays and intersect them with the mesh- For every camera pose do the following: the camera pose is given in the original frame of the scan: if reference scan in reference coordinates, if rescan in rescan coordinates. To do the computations the designdecision is the following:  transform the "current" mesh into the coordinate system of the "new mesh". in this coordinate system you can then compare the two things. Compare the object of the intersection of the current mesh with the one of the new mesh

In [3]:
#for a given scan returns the reference, if the scan is reference returns the reference
def get_reference_id(data_dir,scan_id):
    #refscan of a reference is the reference
    if scan3r.is_reference(data_dir, scan_id):
        return scan_id
    
    #open the 3Rjson file
    dir_path = osp.join(data_dir,"files","3RScan.json")
    with open(dir_path, 'r') as file:
        data = json.load(file)

    for item in data:
        if any(scan_id in scan["reference"] for scan in item["scans"]):
            return item["reference"]

    return None


print("get the reference of", get_reference_id(data_dir,scan_id))

get the reference of 10b1795b-3938-2467-88fe-b10cad8913f8


In [19]:
#transform the mesh based on the procided matrix, mesh is given as ply file
def transform_mesh(curr_scan_id, matrix):
    #read the file
    mesh_path = osp.join(data_dir,"scenes", curr_scan_id, 'labels.instances.annotated.v2.ply')
    with open(mesh_path, 'rb') as file:
        plydata = PlyData.read(file)
    #transform the points into the correct coordinate system using homog coordinates
    points = np.stack((plydata['vertex']['x'], plydata['vertex']['y'], plydata['vertex']['z'])).transpose()
    points4f = np.insert(points, 3, values=1, axis=1)
    points = points4f * matrix

    #put them back into 3d points
    plydata['vertex']['x'] = np.asarray(points[:,0]).flatten()
    plydata['vertex']['y'] = np.asarray(points[:,1]).flatten()
    plydata['vertex']['z'] = np.asarray(points[:,2]).flatten()

    #needs to be stored somewho
    output_path = osp.join(data_dir,"tmpfiles", curr_scan_id, 'transformation.ply')
    directory = osp.dirname(output_path)
    os.makedirs(directory, exist_ok=True)

    plydata.write(output_path)

    return plydata


In [10]:
# transform the current mesh into the coordinate system of the assuming the original meshes in original coordinate systems
def transform_to_coordinates(data_dir, curr_scan_id, new_scan_id):

    #get the path to the matricies of each scan_id for transformation of rescan to reference
    path = osp.join(data_dir,"files", "3RScan.json")
    rescan2ref_all_id = scan3r.read_transform_mat(path)

    #either both reference or the same id -> same coordinate system already
    if curr_scan_id == new_scan_id:
        mesh_path = osp.join(data_dir,"scenes", curr_scan_id, 'labels.instances.annotated.v2.ply')
        with open(mesh_path, 'rb') as file:
            plydata = PlyData.read(file)
        return plydata
    

    # current is reference, new is rescan -> simple transformation 
    elif scan3r.is_reference(data_dir, curr_scan_id):
        #get the rescantore for the new scan id
        rescan2ref = rescan2ref_all_id[new_scan_id]
        rescan2ref = rescan2ref.reshape(4,4)
        ref2rescan = np.linalg.inv(rescan2ref)
        return transform_mesh(curr_scan_id, ref2rescan)
    
    # current is rescan, new is reference -> simple transformation
    elif scan3r.is_reference(data_dir, new_scan_id):
        rescan2ref = rescan2ref_all_id[curr_scan_id]
        rescan2ref = rescan2ref.reshape(4,4)
        return transform_mesh(curr_scan_id, rescan2ref)
    
    #both are rescans but different rescans
    else:
        assert(scan3r.is_rescan(data_dir, curr_scan_id))
        assert(scan3r.is_rescan(data_dir, new_scan_id))

        #transformation matrix from current rescan coord to ref coordinates
        rescan2ref_1 = rescan2ref_all_id[curr_scan_id]
        rescan2ref_1 = rescan2ref_1.reshape(4,4)

        #transformation matrix from ref coord to new rescan coordinates
        rescan2ref_2 = rescan2ref_all_id[new_scan_id]
        rescan2ref_2 = rescan2ref_2.reshape(4,4)
        ref2rescan_2 = np.linalg.inv(rescan2ref_2)

        #combine the 2 transformations first_1 then _2 so from right to left matrix multiplication
        rescan2rescan = ref2rescan_2 @ rescan2ref_1
        return transform_mesh(curr_scan_id, rescan2rescan)


In [25]:
#sanity check if the transformations actually do what they are supposed to do using visuals

"""
Curr Mesh preparation and display adding
"""
pathToMesh = osp.join(data_dir,"scenes", curr_scan_id, "labels.instances.annotated.v2.ply")
curr_mesh = o3d.io.read_triangle_mesh(pathToMesh)
# Visualize the mesh

#check if it has colours
if not curr_mesh.has_vertex_colors():
    print("Mesh does not have vertex colors")
    exit()

#make sure the colours are a oke
colors = np.asarray(curr_mesh.vertex_colors)
if np.max(colors) > 1.0:  # Assuming colors are in the range [0, 255]
        colors /= 255.0

#bruh swap the channels bc the colours from gt didn't match like zurias hint -> open: wtf does that mean for the rest
colors = np.asarray(curr_mesh.vertex_colors)
colors = colors[:, [2, 1, 0]]  # Swap red and blue channels
curr_mesh.vertex_colors = o3d.utility.Vector3dVector(colors)


"""
Transform the current mesh into the coordinate system of the new mesh
"""

transformed_mesh = transform_to_coordinates(data_dir, curr_scan_id, new_scan_id)
pathToMesh = osp.join(data_dir,"tmpfiles", curr_scan_id, "transformation.ply")
transformed_mesh = o3d.io.read_triangle_mesh(pathToMesh)
#check if it has colours
if not transformed_mesh.has_vertex_colors():
    print("Mesh does not have vertex colors")
    exit()

#make sure the colours are a oke
colors = np.asarray(transformed_mesh.vertex_colors)
if np.max(colors) > 1.0:  # Assuming colors are in the range [0, 255]
        colors /= 255.0

#bruh swap the channels bc the colours from gt didn't match like zurias hint -> open: wtf does that mean for the rest
colors = np.asarray(transformed_mesh.vertex_colors)
colors = colors[:, [2, 1, 0]]  # Swap red and blue channels
transformed_mesh.vertex_colors = o3d.utility.Vector3dVector(colors)



"""
New_scan_id mesh display
"""
pathToMesh = osp.join(data_dir,"scenes", new_scan_id, "labels.instances.annotated.v2.ply")
new_mesh = o3d.io.read_triangle_mesh(pathToMesh)
# Visualize the mesh

#check if it has colours
if not new_mesh.has_vertex_colors():
    print("Mesh does not have vertex colors")
    exit()

#make sure the colours are a oke
colors = np.asarray(new_mesh.vertex_colors)
if np.max(colors) > 1.0:  # Assuming colors are in the range [0, 255]
        colors /= 255.0

#bruh swap the channels bc the colours from gt didn't match like zurias hint -> open: wtf does that mean for the rest
colors = np.asarray(new_mesh.vertex_colors)
colors = colors[:, [2, 1, 0]]  # Swap red and blue channels
new_mesh.vertex_colors = o3d.utility.Vector3dVector(colors)




##visualize everything
vis = o3d.visualization.Visualizer()
vis.create_window(width=800, height=600)

vis.add_geometry(curr_mesh)
vis.add_geometry(transformed_mesh)
vis.add_geometry(new_mesh)

vis.run()
vis.destroy_window()
